Cleaning a student score data : -create a year column -re-arrange column values -create a data with student id with unique teacher id for each subject.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import warnings
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')

In [12]:
df_22_23 = pd.read_excel('/content/drive/MyDrive/Data project folder/COLONIAL_E3_Data Request Form 2023_Rev Dec 2023 Justin Bittner.xlsx',
                         sheet_name='Student')
df_22_23.head()

,Year,Student ID,Grade,Building,Race/Ethnicity,FRL Status,ELL Status,SPED Status,General Education Teacher ID,ELA Teacher ID,Math Teacher ID,ELA Standardized Test,Math Standardized Test,ELA Score,ELA Proficiency Level,Math Score,Math Proficiency Level
0,2023,577830,3,Downie (Carrie) Elementary School,Hispanic or Latino of any race,NaN,Y,N,183032,183032,183032,SMARTER,SMARTER,2285,1,2313.0,1.0
1,2023,554991,3,Downie (Carrie) Elementary School,Black or African American,NaN,N,N,199432,199432,199432,SMARTER,SMARTER,2485,3,2404.0,2.0
2,2023,554970,3,Downie (Carrie) Elementary School,Hispanic or Latino of any race,NaN,N,N,199432,199432,199432,SMARTER,SMARTER,2238,1,2156.0,1.0
3,2023,516764,3,Downie (Carrie) Elementary School,Black or African American,NaN,N,N,199432,199432,199432,SMARTER,SMARTER,2433,3,2404.0,2.0
4,2023,346531,3,Downie (Carrie) Elementary School,Black or African American,NaN,N,N,183032,183032,183032,SMARTER,SMARTER,2235,1,2293.0,1.0


In [7]:
df_22_23.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5073 entries, 0 to 5072
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Year                          5073 non-null   int64  
 1   Student ID                    5073 non-null   int64  
 2   Grade                         5073 non-null   int64  
 3   Building                      5073 non-null   object 
 4   Race/Ethnicity                5073 non-null   object 
 5   FRL Status                    0 non-null      float64
 6   ELL Status                    5073 non-null   object 
 7   SPED Status                   5073 non-null   object 
 8   General Education Teacher ID  2206 non-null   object 
 9   ELA Teacher ID                4876 non-null   object 
 10  Math Teacher ID               4881 non-null   object 
 11  ELA Standardized Test         5073 non-null   object 
 12  Math Standardized Test        5073 non-null   object 
 13  ELA

In [13]:
df_22_23 = df_22_23.rename(columns={'Building': 'School'})

In [14]:
df_22_23['Academic Year'] = '2022-2023'
df_22_23['District'] = 'COLONIAL'

df_22_23=df_22_23[['Student ID', 'Academic Year',
                   'District', 'School',
                   'Grade', 'Race/Ethnicity',
                   'ELA Teacher ID', 'ELA Score', 'ELA Proficiency Level',
                   'Math Teacher ID', 'Math Score','Math Proficiency Level']]


In [15]:
df_22_23.loc[df_22_23['Student ID'].duplicated()]

,Student ID,Academic Year,District,School,Grade,Race/Ethnicity,ELA Teacher ID,ELA Score,ELA Proficiency Level,Math Teacher ID,Math Score,Math Proficiency Level


In [16]:
#select the columns with ela score and proficiency.
df_22_23_ela=df_22_23[[
    'Student ID','Academic Year','District', 'Grade','Race/Ethnicity',
    'ELA Teacher ID','ELA Score','ELA Proficiency Level','School']]
#add column
df_22_23_ela['Course']='ELA'
#drop null values
df_22_23_ela = df_22_23_ela.dropna(
    how='any',
    subset=['ELA Teacher ID', 'ELA Score', 'ELA Proficiency Level'])
df_22_23_ela=df_22_23_ela.rename(
    columns={'ELA Teacher ID':'Teacher ID',
             'ELA Score':'Score', 'ELA Proficiency Level':'Proficiency Level'})
#change the type of score from float to integer
df_22_23_ela['Score'] = df_22_23_ela['Score'].astype('int')


In [17]:
df_22_23_math=df_22_23[[
    'Student ID','Academic Year','District', 'Grade','Race/Ethnicity',
    'Math Teacher ID','Math Score','Math Proficiency Level','School']]
df_22_23_math['Course']='MATH'
df_22_23_math = df_22_23_math.dropna(
    how='any',
    subset=['Math Teacher ID', 'Math Score', 'Math Proficiency Level'])
df_22_23_math=df_22_23_math.rename(
    columns={'Math Teacher ID':'Teacher ID',
             'Math Score':'Score',
             'Math Proficiency Level':'Proficiency Level'})

df_22_23_math['Score'] = df_22_23_math['Score'].astype('int')

In [18]:
#append both ela and maths score
df_22_23 = pd.concat([df_22_23_ela, df_22_23_math], axis=0)

In [19]:

df_c1 = (df_22_23.groupby('Student ID')['District'].count().reset_index())
assert len(df_c1[df_c1['District']>2]) == 0

In [21]:
df_22_23.to_csv('/content/drive/MyDrive/Data project folder/Colonial Student File 23_clean.csv',
                index=False, encoding='utf-8')